# 🎯 Hybrid Grasp Detection v10

## New in v10
| Feature | Description |
|---------|-------------|
| **🧠 Depth-Enhanced Scoring** | ใช้ Depth เต็มรูปแบบสำหรับ quality scoring |
| **🎨 Depth Segmentation** | แยกวัตถุด้วย depth map |
| **📊 Quality Heatmap** | แสดง grasp quality overlay |
| **🔬 Hybrid PCA+Depth** | รวม PCA geometry กับ depth features |

## 1️⃣ Imports

In [1]:
import sys
import cv2
import numpy as np
import time
import socket
import serial
import torch
from collections import deque
from ultralytics import YOLO
from scipy import ndimage

sys.path.append('Depth-Anything-V2')

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
print("✓ Imports")

PyTorch: 2.9.1+cpu
CUDA: False
✓ Imports


## 2️⃣ Hardware Configuration

In [2]:
ROBOT_IP = '192.168.1.6'
ESP32_PORT = 'COM9'
ESP32_BAUDRATE = 115200
CAMERA_ID = 2
"""
HOMOGRAPHY_MATRIX = np.array([
    [0.005703976266962427, -0.3265299161278153, 88.58634169557483],
    [-0.47704058225560797, 0.015355046930804153, 172.0941543570439],
    [-0.00029949919510557677, 0.00018728182448344945, 1.0],
], dtype=np.float32)"""

HOMOGRAPHY_MATRIX = np.load('homography_matrix.npy')

print("✓ Hardware config")

✓ Hardware config


## 3️⃣ Configuration

In [3]:
# === CALIBRATED VALUES ===
PIXELS_PER_MM = 2.7703
DEPTH_Z_SCALE = 45.3800
CAMERA_HEIGHT_MM = 630

# === Robot R Rotation ===
ROBOT_R_OFFSET = -25.55

# === Z Heights ===
Z_FLOOR = -64
Z_SAFE = -40
Z_APPROACH = -55

# === Drop Position ===
DROP_POS = (-253.07, 115.17, -17.07, -62.78)

# === Gripper ===
GRIPPER_SERVO_OPEN_ANGLE = 22
GRIPPER_SERVO_CLOSE_ANGLE = 96
GRIPPER_MAX_WIDTH_MM = 54
GRIPPER_MIN_WIDTH_MM = 0
GRIPPER_OPEN_MARGIN_MM = 5
GRIPPER_GRIP_MARGIN_MM = 5

# === Detection ===
MIN_OBJECT_AREA = 1000
YOLO_CONFIDENCE = 0.25

# === Depth Model ===
DEPTH_MODEL_PATH = 'Depth-Anything-V2/checkpoints/depth_anything_v2_vits.pth'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# === v10: Depth Quality Scoring ===
DEPTH_EDGE_WEIGHT = 0.3      # น้ำหนักสำหรับขอบวัตถุชัดเจน
DEPTH_CENTER_WEIGHT = 0.4    # น้ำหนักสำหรับ grasp ที่กลางวัตถุ
DEPTH_FLATNESS_WEIGHT = 0.3  # น้ำหนักสำหรับพื้นผิวเรียบ

# === Grid Display ===
SHOW_FRAME_GRID = False
SHOW_OBJECT_GRID = True
SHOW_DEPTH_OVERLAY = True
FRAME_GRID_SIZE_MM = 20
OBJECT_GRID_SIZE_MM = 5

print("✓ Configuration (v10 with Depth-Enhanced Scoring)")

✓ Configuration (v10 with Depth-Enhanced Scoring)


## 4️⃣ Load Models

In [4]:
from depth_anything_v2.dpt import DepthAnythingV2

model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
}

print("Loading DepthAnything V2...")
depth_model = DepthAnythingV2(**model_configs['vits'])
depth_model.load_state_dict(torch.load(DEPTH_MODEL_PATH, map_location='cpu', weights_only=True))
depth_model = depth_model.to(DEVICE).eval()
print(f"✅ Depth model on {DEVICE}")

print("Loading YOLOv8...")
yolo_model = YOLO('yolov8n.pt')
print("✅ YOLO loaded")

xFormers not available
xFormers not available


Loading DepthAnything V2...
✅ Depth model on cpu
Loading YOLOv8...
✅ YOLO loaded


## 5️⃣ Classes Definition

In [5]:
class SmartGripperController:
    CALIB_ANGLES = np.array([22, 30, 40, 50, 60, 70, 80, 90, 96])
    CALIB_WIDTHS = np.array([54.0, 52.0, 48.0, 40.0, 32.0, 23.0, 12.0, 3.0, 0.0])
    
    def __init__(self, port='COM9', baudrate=115200):
        self.port = port
        self.baudrate = baudrate
        self.serial = None
        self.target_width = None
        
    def connect(self):
        try:
            self.serial = serial.Serial(self.port, self.baudrate, timeout=2)
            time.sleep(2)
            print(f"✅ Gripper on {self.port}")
            return True
        except Exception as e:
            print(f"❌ {e}")
            return False
    
    def disconnect(self):
        if self.serial: self.serial.close()
    
    def send_command(self, cmd):
        if self.serial:
            self.serial.write((cmd + '\n').encode())
            time.sleep(0.3)
    
    def mm_to_angle(self, width_mm):
        width = max(0.0, min(54.0, width_mm))
        return int(round(np.interp(width, self.CALIB_WIDTHS[::-1], self.CALIB_ANGLES[::-1])))
    
    def open_for_object(self, width_mm):
        self.target_width = width_mm
        open_w = min(54.0, width_mm + GRIPPER_OPEN_MARGIN_MM)
        angle = self.mm_to_angle(open_w)
        print(f"🦾 Open: {open_w:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
    
    def grip_object(self, width_mm):
        grip_w = max(0.0, width_mm - GRIPPER_GRIP_MARGIN_MM)
        angle = self.mm_to_angle(grip_w)
        print(f"🦾 Grip: {width_mm:.1f}mm - {GRIPPER_GRIP_MARGIN_MM}mm = {grip_w:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
    
    def release(self):
        open_w = min(54.0, (self.target_width or 30) + 10)
        self.send_command(f'G{self.mm_to_angle(open_w)}')
        self.target_width = None

print("✓ Gripper class")

✓ Gripper class


In [6]:
class DobotControllerTCP:
    def __init__(self, homography_matrix=None, r_offset=-25.55):
        self.dashboard_port = 29999
        self.sock = None
        self.H = homography_matrix
        self.r_offset = r_offset
        
    def connect(self, ip):
        try:
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock.settimeout(5)
            self.sock.connect((ip, self.dashboard_port))
            self.send_command("ClearError()")
            time.sleep(0.5)
            self.send_command("EnableRobot()")
            time.sleep(4)
            self.send_command("User(1)")
            self.send_command("Tool(1)")
            self.send_command("SpeedFactor(50)")
            print("✅ Robot connected!")
            return True
        except Exception as e:
            print(f"Error: {e}")
            return False

    def send_command(self, cmd):
        if self.sock:
            self.sock.send((cmd + "\n").encode("utf-8"))
            return self.sock.recv(1024).decode("utf-8")

    def home(self):
        print("🤖 HOME...")
        self.send_command("MovJ(-253.07, 115.17, -17.07, -62.78)")
        time.sleep(4)

    def move_to(self, x, y, z, r=0):
        cmd = f"MovJ({x},{y},{z},{r})"
        print(f"   → {cmd}")
        return self.send_command(cmd)
    
    def move_to_and_wait(self, x, y, z, r=0, wait=3):
        self.move_to(x, y, z, r)
        time.sleep(wait)
    
    def joint_move(self, j1=0, j2=0, j3=0, j4=0):
        cmd = f"JointMovJ({j1},{j2},{j3},{j4})"
        print(f"   → {cmd}")
        return self.send_command(cmd)
    
    def joint_move_and_wait(self, j1=0, j2=0, j3=0, j4=0, wait=3):
        self.joint_move(j1, j2, j3, j4)
        time.sleep(wait)

    def pixel_to_robot(self, u, v):
        if self.H is None: return None, None
        pt = np.array([u, v, 1], dtype=np.float32)
        res = np.dot(self.H, pt)
        return res[0]/res[2], res[1]/res[2]
    
    def camera_angle_to_robot_r(self, camera_angle):
        return self.r_offset - camera_angle

print("✓ Robot class")

✓ Robot class


In [7]:
class DepthEnhancedDetector:
    """v10: YOLO + Contour + Depth Segmentation"""
    
    def __init__(self, yolo_model, depth_model, pixels_per_mm, device='cpu'):
        self.yolo = yolo_model
        self.depth_model = depth_model
        self.ppm = pixels_per_mm
        self.device = device
        self.current_depth = None
        self.floor_depth = None
    
    def update_depth(self, frame):
        self.current_depth = self.depth_model.infer_image(frame)
        return self.current_depth
    
    def calibrate_floor(self, frame):
        depth = self.update_depth(frame)
        h, w = depth.shape
        self.floor_depth = np.median(depth[h//3:2*h//3, w//3:2*w//3])
        print(f"✅ Floor depth: {self.floor_depth:.4f}")
        return self.floor_depth
    
    def detect(self, frame):
        objects = []
        results = self.yolo(frame, conf=YOLO_CONFIDENCE, verbose=False)
        
        for r in results:
            for box in r.boxes:
                x1,y1,x2,y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                
                roi = frame[y1:y2, x1:x2]
                if roi.size == 0: continue
                
                gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                if contours:
                    cnt = max(contours, key=cv2.contourArea)
                    cnt = cnt + np.array([x1, y1])
                    rect = cv2.minAreaRect(cnt)
                    cx, cy = int(rect[0][0]), int(rect[0][1])
                    
                    # v10: Get depth features
                    depth_info = self._get_depth_features(x1, y1, x2-x1, y2-y1)
                    
                    objects.append({
                        'bbox': (x1, y1, x2-x1, y2-y1),
                        'center': (cx, cy),
                        'rect': rect,
                        'rect_size': rect[1],
                        'rect_angle': rect[2],
                        'contour': cnt,
                        'conf': conf,
                        'area': cv2.contourArea(cnt),
                        'depth_info': depth_info
                    })
        
        if not objects:
            objects = self._edge_detect_with_depth(frame)
        
        return objects
    
    def _get_depth_features(self, x, y, w, h):
        if self.current_depth is None or self.floor_depth is None:
            return {'height': 0, 'flatness': 0, 'edge_clarity': 0}
        
        # Resize depth to match frame if needed
        dh, dw = self.current_depth.shape
        fh, fw = 480, 640  # Assumed frame size
        scale_x, scale_y = dw/fw, dh/fh
        
        dx, dy = int(x*scale_x), int(y*scale_y)
        dw2, dh2 = int(w*scale_x), int(h*scale_y)
        
        region = self.current_depth[dy:dy+dh2, dx:dx+dw2]
        if region.size == 0:
            return {'height': 0, 'flatness': 0, 'edge_clarity': 0}
        
        # Height from floor
        height = max(0, (np.median(region) - self.floor_depth) * DEPTH_Z_SCALE)
        
        # Flatness (inverse of std dev)
        flatness = 1.0 / (1.0 + np.std(region))
        
        # Edge clarity (gradient magnitude)
        if region.shape[0] > 2 and region.shape[1] > 2:
            gx = cv2.Sobel(region.astype(np.float32), cv2.CV_32F, 1, 0, ksize=3)
            gy = cv2.Sobel(region.astype(np.float32), cv2.CV_32F, 0, 1, ksize=3)
            edge_mag = np.sqrt(gx**2 + gy**2)
            edge_clarity = np.mean(edge_mag)
        else:
            edge_clarity = 0
        
        return {'height': height, 'flatness': flatness, 'edge_clarity': edge_clarity}
    
    def _edge_detect_with_depth(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(cv2.GaussianBlur(gray, (5,5), 0), 50, 150)
        edges = cv2.dilate(edges, np.ones((3,3), np.uint8), iterations=2)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        objects = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > MIN_OBJECT_AREA:
                hull = cv2.convexHull(cnt)
                rect = cv2.minAreaRect(hull)
                x,y,w,h = cv2.boundingRect(hull)
                depth_info = self._get_depth_features(x, y, w, h)
                objects.append({
                    'bbox': (x,y,w,h),
                    'center': (x+w//2, y+h//2),
                    'rect': rect,
                    'rect_size': rect[1],
                    'rect_angle': rect[2],
                    'contour': hull,
                    'area': area,
                    'depth_info': depth_info
                })
        return sorted(objects, key=lambda o: o['area'], reverse=True)
    
    def get_depth_colormap(self, alpha=0.3):
        if self.current_depth is None:
            return None
        depth_norm = cv2.normalize(self.current_depth, None, 0, 255, cv2.NORM_MINMAX)
        depth_color = cv2.applyColorMap(depth_norm.astype(np.uint8), cv2.COLORMAP_JET)
        return depth_color

print("✓ Depth-Enhanced Detector class")

✓ Depth-Enhanced Detector class


In [8]:
class HybridGraspSelector:
    """v10: PCA + Depth-Enhanced Quality Scoring"""
    
    def __init__(self, pixels_per_mm):
        self.ppm = pixels_per_mm
    
    def analyze_object(self, obj, depth_map=None, floor_depth=None):
        """วิเคราะห์วัตถุด้วย PCA + Depth Quality Scoring"""
        cnt = obj.get('contour')
        if cnt is None or len(cnt) < 5:
            return self._fallback_analysis(obj)
        
        # PCA Analysis
        pts = cnt.reshape(-1, 2).astype(np.float64)
        mean = np.mean(pts, axis=0)
        pts_centered = pts - mean
        
        cov = np.cov(pts_centered.T)
        eigenvalues, eigenvectors = np.linalg.eig(cov)
        
        idx = np.argsort(eigenvalues)[::-1]
        eigenvalues = eigenvalues[idx]
        eigenvectors = eigenvectors[:, idx]
        
        major_axis = eigenvectors[:, 0]
        minor_axis = eigenvectors[:, 1]
        
        angle_major = np.degrees(np.arctan2(major_axis[1], major_axis[0]))
        
        projections = np.dot(pts_centered, minor_axis)
        width_px = np.max(projections) - np.min(projections)
        width_mm = width_px / self.ppm
        
        proj_major = np.dot(pts_centered, major_axis)
        length_px = np.max(proj_major) - np.min(proj_major)
        length_mm = length_px / self.ppm
        
        cx, cy = int(mean[0]), int(mean[1])
        grasp_camera_angle = self._normalize_angle(angle_major + 90)
        
        # v10: Calculate depth-enhanced quality score
        depth_info = obj.get('depth_info', {})
        depth_quality = self._calculate_depth_quality(depth_info, depth_map, cx, cy, floor_depth)
        
        grasps = []
        
        # Primary: Narrow side grasp
        if width_mm <= GRIPPER_MAX_WIDTH_MM:
            base_score = 1.0
            final_score = base_score * 0.5 + depth_quality * 0.5
            grasps.append({
                'center': (cx, cy),
                'width_mm': width_mm,
                'camera_angle': grasp_camera_angle,
                'score': final_score,
                'depth_score': depth_quality,
                'type': 'PCA_narrow',
                'axis_info': {'major': major_axis, 'minor': minor_axis}
            })
        
        # Alternative: Long side grasp
        if length_mm <= GRIPPER_MAX_WIDTH_MM:
            base_score = 0.5
            final_score = base_score * 0.5 + depth_quality * 0.5
            grasps.append({
                'center': (cx, cy),
                'width_mm': length_mm,
                'camera_angle': self._normalize_angle(angle_major),
                'score': final_score,
                'depth_score': depth_quality,
                'type': 'PCA_long',
                'axis_info': None
            })
        
        if not grasps:
            return self._fallback_analysis(obj)
        
        return sorted(grasps, key=lambda g: g['score'], reverse=True)
    
    def _calculate_depth_quality(self, depth_info, depth_map, cx, cy, floor_depth):
        """Calculate quality score based on depth features"""
        if not depth_info:
            return 0.5
        
        flatness = depth_info.get('flatness', 0.5)
        edge_clarity = min(1.0, depth_info.get('edge_clarity', 0) / 10.0)
        height = depth_info.get('height', 0)
        
        # Higher objects are usually easier to grasp
        height_score = min(1.0, height / 50.0) if height > 5 else 0.3
        
        # Center accessibility from depth map
        center_score = 0.5
        if depth_map is not None and floor_depth is not None:
            h, w = depth_map.shape
            if 0 <= cy < h and 0 <= cx < w:
                local_depth = depth_map[max(0,cy-5):min(h,cy+5), max(0,cx-5):min(w,cx+5)]
                if local_depth.size > 0:
                    center_height = (np.median(local_depth) - floor_depth) * DEPTH_Z_SCALE
                    center_score = min(1.0, max(0.3, center_height / 50.0))
        
        quality = (
            DEPTH_EDGE_WEIGHT * edge_clarity +
            DEPTH_CENTER_WEIGHT * center_score +
            DEPTH_FLATNESS_WEIGHT * flatness
        )
        
        return min(1.0, max(0.0, quality))
    
    def _fallback_analysis(self, obj):
        rect = obj.get('rect')
        if rect is None:
            return []
        
        (cx, cy), (w, h), angle = rect
        cx, cy = int(cx), int(cy)
        
        if w < h:
            grip_w = w / self.ppm
            grip_a = angle + 90
        else:
            grip_w = h / self.ppm
            grip_a = angle
        
        grasps = []
        if grip_w <= GRIPPER_MAX_WIDTH_MM:
            grasps.append({
                'center': (cx, cy),
                'width_mm': grip_w,
                'camera_angle': self._normalize_angle(grip_a),
                'score': 0.6,
                'depth_score': 0.5,
                'type': 'rect_fallback',
                'axis_info': None
            })
        
        return grasps
    
    def _normalize_angle(self, angle):
        while angle > 90: angle -= 180
        while angle < -90: angle += 180
        return angle

print("✓ Hybrid Grasp Selector (v10)")

✓ Hybrid Grasp Selector (v10)


## 6️⃣ Initialize Components

In [9]:
gripper = SmartGripperController(port=ESP32_PORT, baudrate=ESP32_BAUDRATE)
robot = DobotControllerTCP(homography_matrix=HOMOGRAPHY_MATRIX, r_offset=ROBOT_R_OFFSET)
detector = DepthEnhancedDetector(yolo_model, depth_model, PIXELS_PER_MM, device=DEVICE)
grasp_selector = HybridGraspSelector(PIXELS_PER_MM)
print("✓ Components initialized (v10 Hybrid)")

✓ Components initialized (v10 Hybrid)


---
# 🧪 TEST GRASP MODEL (No Robot)
---

In [10]:
print("="*60)
print("🧪 TEST GRASP MODEL (No Robot Connection)")
print("   Click=Select Object | D=Toggle Depth | Q=Quit")
print("="*60)

test_selected = None
test_grasps = []
show_depth = True

def test_mouse_cb(event, x, y, flags, param):
    global test_selected, test_grasps
    if event == cv2.EVENT_LBUTTONDOWN:
        for obj in param.get('objects', []):
            bx,by,bw,bh = obj['bbox']
            if bx <= x <= bx+bw and by <= y <= by+bh:
                test_selected = obj
                test_grasps = grasp_selector.analyze_object(
                    obj, 
                    detector.current_depth, 
                    detector.floor_depth
                )
                if test_grasps:
                    g = test_grasps[0]
                    print(f"\n📦 Selected: W={g['width_mm']:.1f}mm Angle={g['camera_angle']:.1f}°")
                    print(f"   Score: {g['score']:.2f} (Depth: {g['depth_score']:.2f})")
                    print(f"   Type: {g['type']}")
                break

cap = cv2.VideoCapture(CAMERA_ID)
#cap = cv2.VideoCapture(1)
cv2.namedWindow('Test v10')
callback_data = {'objects': []}
cv2.setMouseCallback('Test v10', test_mouse_cb, callback_data)

# Calibrate floor first
print("\n📷 Press SPACE to calibrate floor...")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    cv2.putText(frame, "SPACE=Calibrate Floor | Q=Skip", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
    cv2.imshow('Test v10', frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord(' '):
        detector.calibrate_floor(frame)
        break
    elif key == ord('q'):
        print("Skipped calibration")
        break

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    frame_count += 1
    if frame_count % 5 == 0:
        detector.update_depth(frame)
    
    detected = detector.detect(frame)
    callback_data['objects'] = detected
    
    display = frame.copy()
    
    # Depth overlay
    if show_depth and detector.current_depth is not None:
        depth_color = detector.get_depth_colormap()
        if depth_color is not None:
            depth_resized = cv2.resize(depth_color, (frame.shape[1], frame.shape[0]))
            display = cv2.addWeighted(display, 0.7, depth_resized, 0.3, 0)
    
    # Draw objects
    for obj in detected:
        x,y,w,h = obj['bbox']
        is_sel = test_selected and obj['center'] == test_selected['center']
        color = (0,0,255) if is_sel else (0,255,0)
        cv2.rectangle(display, (x,y), (x+w,y+h), color, 2)
        
        # Show depth info
        d_info = obj.get('depth_info', {})
        h_mm = d_info.get('height', 0)
        cv2.putText(display, f"H:{h_mm:.0f}mm", (x, y-5), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
    
    # Draw grasps
    if test_selected and test_grasps:
        for i, g in enumerate(test_grasps):
            cx, cy = g['center']
            angle = g['camera_angle']
            is_best = (i == 0)
            color = (0,0,255) if is_best else (0,255,255)
            thick = 3 if is_best else 2
            
            length = 40
            dx = int(length * np.cos(np.radians(angle)))
            dy = int(length * np.sin(np.radians(angle)))
            cv2.line(display, (cx-dx, cy-dy), (cx+dx, cy+dy), color, thick)
            cv2.circle(display, (cx, cy), 5, color, -1)
            
            # Quality bar
            bar_len = int(g['score'] * 50)
            cv2.rectangle(display, (cx+15, cy-5), (cx+15+bar_len, cy+5), color, -1)
            cv2.putText(display, f"{g['score']:.2f}", (cx+70, cy+5), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.35, color, 1)
    
    # Status bar
    cv2.rectangle(display, (0,0), (640,35), (30,30,30), -1)
    status = f"v10 TEST | Obj:{len(detected)} | D=Depth:{show_depth} | Click | Q"
    cv2.putText(display, status, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 1)
    
    cv2.imshow('Test v10', display)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): break
    elif key == ord('d'):
        show_depth = not show_depth
        print(f"Depth overlay: {show_depth}")
    elif key == ord('r'):
        test_selected = None
        test_grasps = []

cap.release()
cv2.destroyAllWindows()
print("\n✅ Test complete")

🧪 TEST GRASP MODEL (No Robot Connection)
   Click=Select Object | D=Toggle Depth | Q=Quit

📷 Press SPACE to calibrate floor...
Skipped calibration

✅ Test complete


---
# 🤖 Connect Hardware (When Ready)
---

In [12]:
gripper.connect()

✅ Gripper on COM9


True

In [18]:
robot.connect(ROBOT_IP)

✅ Robot connected!


True

In [21]:
robot.home()

🤖 HOME...


---
# 🎯 MAIN PICK-AND-PLACE v10
---

In [22]:
selected_object = None
selected_grasp = None
detected_objects = []
current_grasps = []

def mouse_callback(event, x, y, flags, param):
    global selected_object, selected_grasp, current_grasps
    if event == cv2.EVENT_LBUTTONDOWN:
        for g in current_grasps:
            gx, gy = g['center']
            if abs(x-gx) < 20 and abs(y-gy) < 20:
                selected_grasp = g
                robot_r = robot.camera_angle_to_robot_r(g['camera_angle'])
                print(f"\n🎯 Grasp: W={g['width_mm']:.1f}mm → R={robot_r:.1f}° Score={g['score']:.2f}")
                return
        for obj in detected_objects:
            bx,by,bw,bh = obj['bbox']
            if bx <= x <= bx+bw and by <= y <= by+bh:
                selected_object = obj
                current_grasps = grasp_selector.analyze_object(
                    obj, detector.current_depth, detector.floor_depth
                )
                selected_grasp = current_grasps[0] if current_grasps else None
                if selected_grasp:
                    robot_r = robot.camera_angle_to_robot_r(selected_grasp['camera_angle'])
                    print(f"\n📦 Object: {len(current_grasps)} grasps")
                    print(f"   Best: W={selected_grasp['width_mm']:.1f}mm R={robot_r:.1f}° Score={selected_grasp['score']:.2f}")
                break

def draw_grasps(frame, grasps, selected):
    for g in grasps:
        cx, cy = g['center']
        angle = g['camera_angle']
        is_sel = (selected and g == selected)
        color = (0,0,255) if is_sel else ((0,255,0) if g['score']>=0.7 else (0,255,255))
        thick = 3 if is_sel else 2
        
        length = 40
        dx = int(length * np.cos(np.radians(angle)))
        dy = int(length * np.sin(np.radians(angle)))
        cv2.line(frame, (cx-dx, cy-dy), (cx+dx, cy+dy), color, thick)
        cv2.circle(frame, (cx, cy), 5, color, -1)
        
        grip_half = int(g['width_mm'] * PIXELS_PER_MM / 2)
        perp_angle = angle + 90
        px = int(grip_half * np.cos(np.radians(perp_angle)))
        py = int(grip_half * np.sin(np.radians(perp_angle)))
        cv2.line(frame, (cx+px-dx//2, cy+py-dy//2), (cx+px+dx//2, cy+py+dy//2), color, 2)
        cv2.line(frame, (cx-px-dx//2, cy-py-dy//2), (cx-px+dx//2, cy-py+dy//2), color, 2)
        
        if is_sel:
            robot_r = robot.camera_angle_to_robot_r(angle)
            cv2.putText(frame, f"{g['width_mm']:.1f}mm R={robot_r:.0f} Q={g['score']:.2f}", 
                       (cx+10, cy-10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)

"""def pick_with_grasp(obj, grasp):
    cx, cy = grasp['center']
    grip_w = grasp['width_mm']
    camera_angle = grasp['camera_angle']
    robot_r = robot.camera_angle_to_robot_r(camera_angle)
    
    robot_x, robot_y = robot.pixel_to_robot(cx, cy)
    d_info = obj.get('depth_info', {})
    height = d_info.get('height', 0)
    z_grasp = max(Z_FLOOR, min(Z_SAFE, Z_FLOOR + height * 0.5))
    
    print(f"\n🤖 Pick: W={grip_w:.1f}mm Angle={camera_angle:.1f}° → R={robot_r:.1f}° Z={z_grasp:.1f}")
    print(f"   Score: {grasp['score']:.2f} Type: {grasp['type']}")
    
    print("🔄 Safe position...")
    robot.joint_move_and_wait(0, 0, 0, 0, 3)
    
    gripper.open_for_object(GRIPPER_MAX_WIDTH_MM)
    time.sleep(2.5)
    
    robot.move_to_and_wait(robot_x, robot_y, Z_APPROACH, robot_r, 3)
    robot.move_to_and_wait(robot_x, robot_y, z_grasp, robot_r, 2)
    
    gripper.grip_object(grip_w - 8.5)
    time.sleep(2.5)
    
    robot.move_to_and_wait(robot_x, robot_y, Z_SAFE, robot_r, 2)
    robot.move_to_and_wait(*DROP_POS[:3], DROP_POS[3], 3)
    
    gripper.release()
    time.sleep(2.5)
    robot.home()
    print("✅ Complete!")"""

def pick_with_grasp(obj, grasp):
    cx, cy = grasp['center']
    grip_w = grasp['width_mm']
    camera_angle = grasp['camera_angle']
    robot_r = robot.camera_angle_to_robot_r(camera_angle)
    
    robot_x, robot_y = robot.pixel_to_robot(cx, cy)
    d_info = obj.get('depth_info', {})
    height = d_info.get('height', 0)
    z_grasp = max(Z_FLOOR, min(Z_SAFE, Z_FLOOR + height * 0.5))
    
    print(f"\n🤖 Pick: W={grip_w:.1f}mm Angle={camera_angle:.1f}° → R={robot_r:.1f}° Z={z_grasp:.1f}")
    print(f"   Score: {grasp['score']:.2f} Type: {grasp['type']}")
    
    print("🔄 Safe position...")
    robot.joint_move_and_wait(0, 0, 0, 0, 3)
    
    gripper.open_for_object(GRIPPER_MAX_WIDTH_MM)
    time.sleep(3)
    
    # ✅ ลงไปที่ z_grasp โดยตรง (ไม่ผ่าน Z_APPROACH - ป้องกันชนวัตถุสูง)
    robot.move_to_and_wait(robot_x, robot_y, z_grasp, robot_r, 3)
    
    gripper.grip_object(grip_w - 8.7)
    time.sleep(2.5)
    
    # ✅ ไปที่ตำแหน่งวางโดยตรง (ไม่ผ่าน Z_SAFE - ป้องกันชนเมื่อยกของสูง)
    robot.move_to_and_wait(*DROP_POS[:3], DROP_POS[3], 3)
    
    gripper.release()
    time.sleep(2.5)
    robot.home()
    print("✅ Complete!")

# Main loop
cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('Pick v10')
cv2.setMouseCallback('Pick v10', mouse_callback)

# Calibrate floor
print("📷 Press SPACE to calibrate floor...")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    cv2.putText(frame, "SPACE=Calibrate | Q=Skip", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
    cv2.imshow('Pick v10', frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord(' '):
        detector.calibrate_floor(frame)
        break
    elif key == ord('q'): break

frame_count = 0
print("="*60)
print("🎯 PICK v10 (Hybrid PCA + Depth)")
print("Click=Select | SPACE=Execute | D=Depth | Q=Quit")
print("="*60)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    frame_count += 1
    if frame_count % 5 == 0:
        detector.update_depth(frame)
    
    detected_objects = detector.detect(frame)
    
    display = frame.copy()
    if SHOW_DEPTH_OVERLAY and detector.current_depth is not None:
        depth_color = detector.get_depth_colormap()
        if depth_color is not None:
            depth_resized = cv2.resize(depth_color, (frame.shape[1], frame.shape[0]))
            display = cv2.addWeighted(display, 0.7, depth_resized, 0.3, 0)
    
    for obj in detected_objects:
        x,y,w,h = obj['bbox']
        is_sel = (selected_object and obj['center'] == selected_object['center'])
        color = (0,0,255) if is_sel else (0,255,0)
        cv2.rectangle(display, (x,y), (x+w,y+h), color, 2)
        
        d_info = obj.get('depth_info', {})
        cv2.putText(display, f"H:{d_info.get('height',0):.0f}mm", (x, y-5), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
    
    if selected_object and current_grasps:
        draw_grasps(display, current_grasps, selected_grasp)
    
    cv2.rectangle(display, (0,0), (640,35), (30,30,30), -1)
    cv2.putText(display, f"v10 Hybrid | Obj:{len(detected_objects)} | Click | SPACE | D | Q",
               (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 1)
    
    if selected_grasp:
        robot_r = robot.camera_angle_to_robot_r(selected_grasp['camera_angle'])
        cv2.putText(display, f"[{selected_grasp['type']}: W={selected_grasp['width_mm']:.1f}mm R={robot_r:.0f} Q={selected_grasp['score']:.2f} - SPACE]",
                   (10, 470), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
    
    cv2.imshow('Pick v10', display)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): break
    elif key == ord('r'):
        selected_object = None
        selected_grasp = None
        current_grasps = []
    elif key == ord('d'):
        SHOW_DEPTH_OVERLAY = not SHOW_DEPTH_OVERLAY
        print(f"Depth overlay: {SHOW_DEPTH_OVERLAY}")
    elif key == ord('h'):
        robot.home()
    elif key == ord(' ') and selected_object and selected_grasp:
        pick_with_grasp(selected_object, selected_grasp)
        selected_object = None
        selected_grasp = None
        current_grasps = []

cap.release()
cv2.destroyAllWindows()

📷 Press SPACE to calibrate floor...
🎯 PICK v10 (Hybrid PCA + Depth)
Click=Select | SPACE=Execute | D=Depth | Q=Quit

📦 Object: 2 grasps
   Best: W=29.9mm R=-28.5° Score=0.75

🤖 Pick: W=29.9mm Angle=2.9° → R=-28.5° Z=-46.9
   Score: 0.75 Type: PCA_narrow
🔄 Safe position...
   → JointMovJ(0,0,0,0)
🦾 Open: 54.0mm (22°)
   → MovJ(30.816154557895118,-3.4751038588505505,-46.873809814453125,-28.496289222156197)
🦾 Grip: 21.2mm - 5mm = 16.2mm (76°)
   → MovJ(-253.07,115.17,-17.07,-62.78)
🤖 HOME...
✅ Complete!

📦 Object: 2 grasps
   Best: W=21.4mm R=-22.3° Score=0.77

🤖 Pick: W=21.4mm Angle=-3.3° → R=-22.3° Z=-44.1
   Score: 0.77 Type: PCA_narrow
🔄 Safe position...
   → JointMovJ(0,0,0,0)
🦾 Open: 54.0mm (22°)
   → MovJ(-6.56538329809821,41.42992403421328,-44.08769989013672,-22.259695875984352)
🦾 Grip: 12.7mm - 5mm = 7.7mm (85°)
   → MovJ(-253.07,115.17,-17.07,-62.78)
🤖 HOME...
✅ Complete!

📦 Object: 2 grasps
   Best: W=23.1mm R=-47.6° Score=0.72

🤖 Pick: W=23.1mm Angle=22.0° → R=-47.6° Z=-51.6
 

In [ ]:
robot.home()

In [ ]:
gripper.disconnect()
print("✅ Done")